# Emerging Technolgoies

### Libraries 

In [20]:
# Random selections.
import random

# Numerical arrays.
import numpy as np

## Problem 1: Generating Random Boolean Functions


### Define the four constant and balanced functions 

In [21]:
# Constant function 1.
def constant_function_1(a,b,c,d):
  """Takes a single bit but always returns 0."""
  # Ouput will always be 0, regardless of the input.
  # Example output of this function : ('Constant: ', [0, 0, 0, 0, 0, 0, 0, 0])
  return 0

# Balanced function 1.
def balanced_function_1(a,b,c,d):
  """Identity function."""
  # Output will be the same as the input.
  # Example output of this function : ('Balanced: ', [0, 0, 0, 0, 1, 1, 1, 1])
  return a

# Balanced function 2.
def balanced_function_2(a,b,c,d):
  """Not function."""
  # Output will be the opposite of the input.
  # Example output of this function : ('Balanced: ', [1, 0, 1, 0, 1, 0, 1, 0])
  return a ^ b ^ c ^ d

# Constant function 2. 
def constant_function_2(a,b,c,d):
  """Constant 1 function."""
  # Ouput will always be 1, regardless of the input.
  # Example ouput of thsi function : ('Constant: ', [1, 1, 1, 1, 1, 1, 1, 1, 1])
  return 1

### The random_constant_balanced function returns a randomly selected function 

In [22]:
def random_constant_balanced():
  """Randomly returns one of the four functions defined above."""

  # List of potential functions.
  functions = [constant_function_1, balanced_function_1, balanced_function_2, constant_function_2]

  # Choose a random function from the list of fucntions.
  random_function = random.choice(functions)

  # Return the random function.
  return random_function


### fclosure method 

This function returns another function and is instansiated inside the parent function. Its purporse is to hide as much information as possible about the rnadoom function selected.

In [ ]:
def function_closure(n):
    """Return a function that uses an object in the enclosing function."""
    # Get a random function that takes n arguments
    random_function = random_constant_balanced()
    # Define a function that takes n binary arguments and applies them to f_a
    def f(*args):
        if len(args) != n:
            return None
        else:
            # Returns a function that takes 4 arguments and applies them to f_a
            return random_function(*args)
    # Return the closure
    return f

## Problem 2: Classical Testing for Function Type


In [24]:
def determine_constant_balanced(random_function):
    # List to store the results of the function for each input.
    results = []

    # All 16 possible inputs for the function. 
    inputs = [(a, b, c, d) for a in [0, 1] for b in [0, 1] for c in [0, 1] for d in [0, 1]]

    for input in inputs:
        # For each input, call the random function.
        output = random_function(*input)
        # Store the output of the result in the resuls list.
        results.append(output)

        # Count the number of 0s in the results list.
        zeros = results.count(0)
        # Count the number of 1s in the results list.
        ones = results.count(1)

        # Balanced if there are both 0s and 1s in the results list.
        if ones > 0 and zeros > 0:
            return ("Balanced: ", results)

        # With only 16 possible inputs if there are more than 8 1s or 0s then the function is constant.
        if ones > 8 or zeros > 8:
            return ("Constant: ", results)


In [ ]:
# Get a random function using the closure.
function = function_closure(4)

# Determine if the function is constant or balanced.
test = determine_constant_balanced(function)

# Print the result 
print(test)

# function = random_constant_balanced()
# # Determine if the function is constant or balanced.
# test = determine_constant_balanced(function)
# test

('Balanced: ', [0, 0, 0, 0, 0, 0, 0, 0, 1])


### Efficiency of solution

My solution is efficient as it does not loop through all 16 possible inputs to reach a conclusion on whether the function is constant or balanced. A list is defined to store all outputs and the function loops through each input combination and calls the random function on each iteration. The number of 1s and 0s in the output are stored and each round checks the state of the function.

If there are both 0s and 1s in the list of outputs, that is enough to return the function is balanced so there can be as little as two iterations. Through this implementation, all 16 possible inputs are not required to be checked. 



### What is the maximum number of times you need to call f to be 100% certain whether it is balanced?

A function accepting 4 arguments as inputs can only have 16 possible inputs - 8 constant inputs and 8 balanced inputs 

- **The 16 possible inputs**

    (0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1),
    (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)

`Balanced Functions` : A function that returns 0 exactly half of the time and exactly 1 have of the time.

**Minimum**

With a function that accepts 4 argumnets as input the minimum number of times it would need to call the function to determine if it is balanced is 2. If the first time the random function returns 0 and the second time it returns 1, this is enough to classify the function as balanced as a constant function will only ever return 1 or only ever return 0.

- `Example 1`

    - *fucntion call 1 : ouput = 0*
    - *fucntion call 2 : ouput = 1*


**Maximum**

The maximum number of times you would need to call f to determine if it is balanced is 9. With 16 possible input combinations, the function can return all 0s for the first 8 ouputs and on the ninth, return 1. This indicates it is balanced. 

- `Example 2`

    - *fucntion call 1 : ouput = 0*
    - *fucntion call 2 : ouput = 0*
    - *fucntion call 3 : ouput = 0*
    - *fucntion call 4 : ouput = 0*
    - *fucntion call 5 : ouput = 0*
    - *fucntion call 6 : ouput = 0*
    - *fucntion call 7 : ouput = 0*
    - *fucntion call 8 : ouput = 0*
    - *fucntion call 9 : ouput = 1*


The max number of times you need to call f to be 100% certain it is balanced is `9`.


### What is the maximum number of times you need to call f to be 100% certain whether it is constant?

`Constant Functions` : A function always either return 1 or always either returns 0.

**Minimum and Maximum**

With a function that accepts 4 argumnets as input, both the minimum and maximum number of calls it would take to determine if f is a constant function is also `9`. With 16 different input combinations if a function is called 8 times and all the inputs yield 0, without the 9th combination it cannot yet be determined if the function is constant ro balanced. On the 9th function call, if it were to return 0 again, it is constant as more than half of the inputs returned 1.


- `Example`

    - *fucntion call 1 : ouput = 0*
    - *fucntion call 2 : ouput = 0*
    - *fucntion call 3 : ouput = 0*
    - *fucntion call 4 : ouput = 0*
    - *fucntion call 5 : ouput = 0*
    - *fucntion call 6 : ouput = 0*
    - *fucntion call 7 : ouput = 0*
    - *fucntion call 8 : ouput = 0*
    - *fucntion call 9 : ouput = 0*


The max number of times you need to call f to be 100% certain it is consatnt is `9`.


## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit


## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
